In [4]:
# %load_ext autorelaod
# %autoreload 2

import sys
sys.path.append('./../RWKV-v4neo/src')
import torch 
from torch.utils import data
from enwik8_dataset import EnWik8
import json
import argparse
import os
from torch import nn

In [5]:
def load_json(path):
    with open(path, 'r') as f:
        return json.load(f)
def set_environ(env_vars: dict):
    for k, v in env_vars.items():
        os.environ[k] = v

In [6]:
data_path = '/system/user/beck/pwbeck/projects/rwkv/RWKV-LM/data/enwik8'
args = load_json('./debug_args.json')
args = argparse.Namespace(**args)
env_vars = load_json('./debug_environ.json')
set_environ(env_vars)

In [7]:
batch_size = 12

In [8]:
dataset = EnWik8(datafile=data_path)
dataloader = data.DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True, drop_last=True)
args.vocab_size = dataset.vocab_size

KeyboardInterrupt: 

[2/3] c++ -MMD -MF wkv_op.o.d -DTORCH_EXTENSION_NAME=wkv_v1 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /system/apps/userenv/beck/rwkv/lib/python3.10/site-packages/torch/include -isystem /system/apps/userenv/beck/rwkv/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -isystem /system/apps/userenv/beck/rwkv/lib/python3.10/site-packages/torch/include/TH -isystem /system/apps/userenv/beck/rwkv/lib/python3.10/site-packages/torch/include/THC -isystem /system/apps/userenv/beck/rwkv/include -isystem /system/apps/userenv/beck/rwkv/include/python3.10 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++14 -c /system/user/publicwork/beck/projects/rwkv/RWKV-LM/rwkv_playground/cuda/wkv_op.cpp -o wkv_op.o 
[3/3] c++ wkv_op.o wkv_cuda.cuda.o -shared -L/system/apps/userenv/beck/rwkv/lib/python3.10/site-packages/torch/lib -lc10 -lc10_cuda -ltorch_cpu -ltorch_cuda_cu -ltorch_cuda_cpp -ltorch -ltorch

In [ ]:
batch = next(iter(dataloader))
x, y = batch
x.shape, y.shape

(torch.Size([12, 512]), torch.Size([12, 512]))

### RWKV-v4neo model

In [ ]:
from model import RWKV, Block, RWKV_ChannelMix, RWKV_TimeMix
model = RWKV(args)

RWKV_MY_TESTING 


Using /system/user/beck/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /system/user/beck/.cache/torch_extensions/py310_cu117/wkv_512/build.ninja...
Building extension module wkv_512...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module wkv_512...


In [ ]:
output = model(x)

In [ ]:
output.shape

torch.Size([12, 512, 6064])

### Reimplementation of RWKV-v4neo model

In [ ]:
from rwkv_model import RWKV, RWKVConfig

Using /system/user/beck/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /system/user/beck/.cache/torch_extensions/py310_cu117/wkv/build.ninja...
Building extension module wkv...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module wkv...


In [ ]:
config = RWKVConfig(embedding_dim=512,
                    attention_dim=512,
                    ffn_dim=2048,
                    num_blocks=6,
                    vocab_size=dataset.vocab_size, #6064
                    context_len=512)


In [ ]:
mymodel = RWKV(config).to(device='cuda')


In [ ]:
x = x.to(device='cuda')
out = mymodel(x)

In [ ]:
out.shape

torch.Size([12, 512, 6064])

In [2]:
%load_ext autoreload
%autoreload 2
from wkv_kernel import WKV, WKVConfig

In [3]:
wkv = WKV(config=WKVConfig(T_max=23))

Using /system/user/beck/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /system/user/beck/.cache/torch_extensions/py310_cu117/wkv/build.ninja...
Building extension module wkv...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module wkv...


In [8]:
id(wkv)

140275500530560

In [9]:
id(wkv._instance)

140275500530560

In [10]:
wkv.cfg

WKVConfig(T_max=23, cpp_ext_name='wkv', cpp_ext_sources=['cuda/wkv_op.cpp', 'cuda/wkv_cuda.cu'], extra_cuda_cflags=['-res-usage', '--maxrregcount 60', '--use_fast_math', '-O3', '-Xptxas -O3', '-DTmax=23'], device=device(type='cuda'), float_mode='fp32')

In [11]:
wkv2 = WKV()

In [12]:
id(wkv2)

140275500530560

In [13]:
wkv3 = WKV()

In [14]:
type(wkv3)

wkv_kernel.WKV

In [15]:
wkv3.cfg

WKVConfig(T_max=23, cpp_ext_name='wkv', cpp_ext_sources=['cuda/wkv_op.cpp', 'cuda/wkv_cuda.cu'], extra_cuda_cflags=['-res-usage', '--maxrregcount 60', '--use_fast_math', '-O3', '-Xptxas -O3', '-DTmax=23'], device=device(type='cuda'), float_mode='fp32')

In [16]:
type(wkv3)

wkv_kernel.WKV

In [17]:
wkv.cfg

WKVConfig(T_max=23, cpp_ext_name='wkv', cpp_ext_sources=['cuda/wkv_op.cpp', 'cuda/wkv_cuda.cu'], extra_cuda_cflags=['-res-usage', '--maxrregcount 60', '--use_fast_math', '-O3', '-Xptxas -O3', '-DTmax=23'], device=device(type='cuda'), float_mode='fp32')